<a href="https://colab.research.google.com/github/EffiSciencesResearch/ML4G/blob/main/workshops/rlhf/rlhf.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>


This workshop was made by Callum McDougal for ARENA, and serves two purposes:
- Play around RLHF
- Familiarize with new libraries that might be useful for some of your future projects (HuggingFace, Weights&Biases, TRLX)

# Reinforcement Learning from Human Feedback


## Introduction


### Context - Pretraining is not enough

You've seen earlier in the course that we can train very large and performant models like GPT2 using next-token prediction. Such models, prior to any fine-tuning, must be steered carefully with prompts in order to generate useful output. Most language models used in services of any kind today are not only pre-trained models. Rather, we use many training techniques to make them more useful.

RLHF is one of many techniques which can convert a pre-trained model, into a more useful model for practical application.

### Context - RLHF as a naive alignment strategy

The field AI alignment is concerned with aligning AI systems with our desired outcomes. There are many reasons to think that intelligent systems do not, by default, share human values or that whilst training against any objective will lead to reliable, expected outcomes being produced by AI systems. Nevertheless, training AI systems to produce outcomes that humans prefer over outcomes which they don't seems to be a concrete step towards AI alignment, which we can build on later.

Thus we get the core idea of RLHF as an alignment strategy. We care about outcomes, so we provide the AI feedback based on what we think likely outcomes of it's actions are and update it to produce good outcomes according to our preferences.

For more detail on RLHF, see Paul Christiano's blog post [here](https://www.alignmentforum.org/posts/vwu4kegAEZTBtpT6p/thoughts-on-the-impact-of-rlhf-research#The_case_for_a_positive_impact).


### What is RLHF?

Reinforcement Learning with Human Feedback (RLHF) is a RL technique where the rewards issued by a reward model, which is itself trained from labelled data from a human operator. Often, it can be hard to specify the reward function $R : S \times A \to \mathbb{R}$ that the environment uses to issue reward to the agent, so we ask a human instead to reward/punish the agent based on the action it took. [OpenAI](https://openai.com/research/learning-from-human-preferences) uses RLHF to adjust the behaviour of models to desirable behaviour, but this can also incentivise the agent to hack the reward signal (by taking actions that look good to the human, or influencing the human to always give good rewards.)

One should note that in the framework of RLHF, the environment only has one state, and the model that we are trying to fine-tune with RLHF no longer needs to "plan ahead", so in this sense it is closer to a bandit problem than the MDPs we saw in previous days.

### Why does it matter?

RLHF (at the moment) is a successful method of nudging large language models towards desired behaviour when that behaviour is difficult to write as an algorithm.

For chess, it's easy to evaluate whether an agent won/lost the game, so we can reward that directly. For text generation, it can be hard to formally specify
what we mean by harmful or abusive text. One could have simple proxies like a filter to encourage/discourge use of particular words, and use that
to train against, but it's very easy to construct harmful text such that no particular word in the sentence would be classed as offensive:
"I would love to eat your pet puppy" contains no offensive words, even though the semantic meaning of the entire sentence is quite offensive.
A simple proxy for offensiveness might even rate this as a positive statement, as it contains "nice" words like *love* and *puppy*.

However, samples from humans are expensive and slow. Even running a single batch of examples through the model could take a long time
if we need a human to give a scalar reward for each action chosen by the model. So, the solution is to collect a lot of data from a human
(a set of (observation, action, reward) tuples), train a reward model on this data, and then use the reward model as the reward function.


### How does RLHF work in practice?

RLHF involves 3 stages:

1. We pretrain a language model (LM) using existing supervised learning techniques.
2. We gather labelled data from humans, and train a reward model that will act as a proxy for the human's rewards.
3. We fine-tuning the LM with reinforcement learning.

#### 1. Pretraining

Since reinforcement learning is very sample inefficient, it is unreasonable to expect to be able to train a language model from scratch using online learning. Rather, we must start with an existing pre-trained model and then fine-tune it.

We will be using GPT-2-small as our base model to finetune.

<img src="https://raw.githubusercontent.com/jbloomAus/ARENA_2.0-RLHF/main/media/pretraining.png" width="500">

#### 2. The Reward Model

The reward model is used to assign a reward to any given output of the model during training.
Rather than have reward be a simple function of the state of the world (as for RL environments like CartPole),
the reward model assigns a reward to a given piece of text.
The reward model acts like a text classifier, rewarding "good" pieces of text, and punishing "bad" text.

The reward model is trained on a set of prompts, hand labelled by humans into "good" and "bad".
This is then used to train the reward model, to act as a stand-in for the human during the fine-tuning stage.

The model acts as a mapping between arbitrary text and human preferences.

<img src="https://raw.githubusercontent.com/jbloomAus/ARENA_2.0-RLHF/main/media/reward-model.png" width="700">

#### 3. Fine-Tuning with Reinforcement Learning

Finally, given some reward model and some pre-trained model, we can use an algorithm such as PPO to reward the model for producing prompt completions when the reward model predicts the completion to be preferable.

In the standard RL framework, the agent recieves a reward on every timestep during interaction.
Here, the "observation" that the agent receives is a textual prompt, and the "action" the agent takes is the choice of words
to complete the prompt. The reward model then assigns a reward based on the prompt together with the completion from the agent,
which is then used to compute the loss, and update the weights of the model.

<img src="https://raw.githubusercontent.com/jbloomAus/ARENA_2.0-RLHF/main/media/rlhf.png" width="800">

### How does RLHF differ from PPO?

- No "environment". RLHF operates on text completions made by the pre-trained generative model.
- Reward Model. Reward itself is generated by the reward model which itself must be trained.
- Adding a Value Head. We add a value head to the policy/LM architecture so that we have both an actor and a critic for PPO.
- KL Divergence penalty. The KL divergence term penalizes the RL policy from moving substantially away from the initial pretrained model with each training batch, to ensure we maintain coherent outputs, and the fine-tuned model avoids generating text that overfits to what the reward model is looking for.

#### Aside - value heads

The "actor" in our PPO setup is the GPT model. We get the "critic" by adding a **value head** to the GPT architecture - i.e. you stick a classifier to GPT2 and train that as our value function.

For an example, see the source code for AutoModelForCausalLMWithValueHead in the [TRLX github](https://github.com/CarperAI/trlx/blob/main/trlx/models/modeling_ppo.py). This gives us an autoregressive transformer which has 2 outputs: one corresponding to the standard next token prediction objective, and one which sticks a classifier on the end to get a value function. It does this by adding `self.v_head`, a function which reads from the final value of the residual stream in GPT (which stores some compressed embedding of the prompt), and extracts a value function from this representation. You can think of this as a kind of feature extraction, analogous to the feature extraction that we implemented with our ResNet models in the first week.

The TRLX library we'll be working with today handles all of this under the hood. However, you should definitely have a poke around this library to get a feel for how it works.

#### Aside - KL divergence term

**Note** - the KL div penalty is not the same as the version of PPO which uses a KL div penalty term in the surrogate objective function. The first one is a feature of the RLHF setup; it makes sure we don't get too far from the original model (i.e. it's static throughout training, used to constrain how much we change from the original model by the end). The second one is a feature of PPO setup; it makes sure we don't make huge updates from where we were before the last training step (i.e. it's a moving target, used to constrain how much we change each step).

The KL div term we use in RL heavily penalises our new model when it outputs something which **would have low probability in the original model.** This is related to the reason RLHF'ed models are sometimes described as ["lobotomized"](https://twitter.com/repligate/status/1640488734192726018) - they converge to a subset of the kinds of outputs that our original model might have had, meaning they lose some of the variance and creativity of the original model.

## Optionnal readings for RLHF

* [Fine-Tuning Language Models from Human Preferences](https://arxiv.org/abs/1909.08593) (paper)
* [Learning to summarize from human feedback](https://arxiv.org/abs/2009.01325) (paper)
* [AI safety via debate](https://openai.com/research/debate) (OpenAI blog post)
* [Thoughts on the impact of RLHF research](https://www.alignmentforum.org/posts/vwu4kegAEZTBtpT6p/thoughts-on-the-impact-of-rlhf-research), by Paul Christiano

## Content & Learning Objectives


#### 1️⃣ Prompt Dataset & Reward Model

In the first section, we'll get set up with the prompt dataset and reward model we'll be using for the rest of the exercises.

> ##### Learning objectives
>
> * Learn about the BERT transformer model and how it can be used for sentiment analysis
> * Load datasets from Huggingface and break them up into prompts
> * Generate text from Huggingface models
> * Output positive sentiments from models in vanilla PyTorch and Huggingface pipelines

#### 2️⃣ Using RLHF for Finetuning

In the second section, we'll finetune a model pre-trained on the IMDB dataset using RLHF to generate positive reviews.

> ##### Learning objectives
>
> - Learn about TRLX and how it can be used
> - Using RLHF to improve sentiment of GPT2 produced Movie Reviews

## Setup

In [ ]:
!pip install datasets transformers trl

In [ ]:
import os

import json
import sys
import math
import gc
from pathlib import Path
import torch
from datasets import load_dataset
from transformers import (
    pipeline,
    AutoTokenizer,
    AutoModelForCausalLM,
    AutoModelForSequenceClassification,
)
from transformers.models.bert.modeling_bert import BertForMaskedLM
import logging
from typing import cast, Any, List, Optional, Union, Tuple

import trl

# 1️⃣ Prompt Dataset & Reward Model


> ##### Learning objectives
>
> * Learn about the BERT transformer model and how it can be used for sentiment analysis
> * Load datasets from Huggingface and break them up into prompts
> * Generate text from Huggingface models
> * Output positive sentiments from models in vanilla PyTorch and Huggingface pipelines

## Background - BERT

In the transformers chapter, we only worked with autoregressive transformers like GPT2. Here, we'll work with BERT, a well-known **bidirectional transformer**.

BERT predates GPT2 slightly (it was released in 2018, one year after the seminal "Attention is all you need" paper). It was the next in a proud tradition of naming transformers after muppets (no, [that's](https://arxiv.org/pdf/1910.13034.pdf) [not](https://arxiv.org/pdf/1904.09223.pdf) [a](https://arxiv.org/pdf/1905.12616.pdf) [joke](https://arxiv.org/pdf/1906.01604.pdf)). It has bidirectional attention, meaning we don't apply masking to the attention patterns - information can flow backwards and forwards in the model. BERT is usually used for classification tasks, such as sentiment analysis.

### How is BERT trained?

The architecture is similar to GPT, although the "core BERT" model doesn't have an unembedding (i.e. the output has shape `(batch, seq_len, d_model)`).

BERT is trained on two kinds of tasks: **next sentence prediction** (NSP) and **masked language modelling** (MLM).

* In MLM, we take a sequence and replace some of its tokens with a special `[MASK]` token, then train the model to predict the original token.
* In NSP, we take two sentences, and train the model to predict whether the second sentence follows the first (we do this by adding a small classifier at the end of BERT, which just reads from the final value of the residual stream at the zeroth sequence position, which is a special classification token `[CLS]`).

Importantly, **both of these two tasks require the model to learn some kind of compressed representation of the input sequence** in its residual stream.

### How do we turn BERT into a classifier?

We usually stick a classification head onto the end of the "core BERT architecture" at the `[CLS]` token, then take the pretrained model and fine-tune it on a classification task. If pretraining has been successful, the model will have learned some kind of compressed representation of the input sequence in its residual stream, and the classifier will be doing something like feature extraction.

In the RLHF exercises you'll be taking advantage of BERT's ability to be used as a classifier, but for now we'll have a look at how BERT does at masked language modelling.

### Exercise - load BERT, and play around with it

```c
Difficulty: 🟠⚪⚪⚪⚪
Importance: 🟠🟠🟠⚪⚪

You should spend up to 10-15 minutes on this exercise.
```

We're going to use a HuggingFace tokenizer for now to encode text into a sequence of tokens that our model can use. The tokenizer has to match the model - our model was trained with the `bert-base-cased` tokenizer which is case-sensitive. If you tried to use the `bert-base-uncased` tokenizer which is case-insensitive, it wouldn't work at all.

Check out `tokenizer.vocab` to get an idea of what sorts of strings are assigned to tokens. In WordPiece, tokens represent a whole word unless they start with `##`, which denotes this token is part of a word.

You can also check out `tokenizer.special_tokens_map`. The strings here are mapped to tokens which have special meanings - for example `tokenizer.mask_token`, which is the literal string '[MASK]', is converted to `tokenizer.mask_token_id`, equal to 103.

**Play around with this model**, until you get a sense of how it works. What kind of interesting completions can you find? Can BERT solve the IOI task? Can it do basic arithmetic?


In [ ]:
bert = BertForMaskedLM.from_pretrained("bert-base-cased")
bert_tokenizer = AutoTokenizer.from_pretrained("bert-base-cased")

In [ ]:
def predict(model: BertForMaskedLM, tokenizer: AutoTokenizer, text: str, k=15) -> List[List[str]]:
    """
    Return a list of k strings for each [MASK] in the input.
    """

    # Make sure we're in eval mode
    model.eval()

    # Tokenizer returns a bunch of special BERT-specific things, we just want input ids
    input_ids = tokenizer(text, return_tensors="pt")["input_ids"]

    # Get top predictions at all the places we masked
    out = model(input_ids).logits
    preds = out[input_ids == tokenizer.mask_token_id]
    tops = preds.topk(k, dim=-1).indices

    return [[tokenizer.decode(t) for t in mask] for mask in tops]


your_text = "The Answer to the Ultimate Question of Life, The Universe, and Everything is [MASK]."
predictions = predict(bert, bert_tokenizer, your_text)
print("Model predicted: \n", "\n".join(map(str, predictions)))

## IMDB dataset


First, load in the IMDB user reviews dataset. Documentation about the IMDB dataset can be found here: https://huggingface.co/datasets/imdb. We want to use both the train and test splits to collect prompts.

In [ ]:
imdb = load_dataset("imdb", split="train+test")

### Exercise - Figure out the positive-negative review split in the dataset

```c
Difficulty: 🟠⚪⚪⚪⚪
Importance: 🟠🟠⚪⚪⚪

You should spend up to 5 minutes on this exercise.
```

The positive-negative review split will tell us the distribution of sentiments our model will output out of the box. Write a function to print out the number of samples for each label.

You should review the [documentation page](https://huggingface.co/datasets/imdb) to know more what the dataset looks like.

<details>
<summary><b>Hint</b> (click to expand)</summary>
- You can access specific columns of the dataset using the `dataset['column_name']` syntax.
- You can use the `.count(value)` method on a column to count the number of occurences of a value in that column.
</details>

In [ ]:
def label_split(dataset) -> Tuple[int, int]:
    ...

nb_positive, nb_negative = label_split(imdb)

print(f"Number of positive reviews: {nb_positive}")
print(f"Number of negative reviews: {nb_negative}")

<details>
<summary>Solution</summary>


```python
def label_split(dataset) -> Tuple[int, int]:
    positive_samples = dataset['label'].count(1)
    negative_samples = dataset['label'].count(0)

    return postive_samples, negative_samples
```
</details>


### Exercise - Create a set of prompts

```c
Difficulty: 🟠🟠⚪⚪⚪
Importance: 🟠🟠🟠⚪⚪

You should spend up to ~10 minutes on this exercise.
```

A prompt to the model can look like "Today was not fun ", "In the event of " or "Mary gave John a ". These prompts will serve as the starting point for model generations during the RLHF process.

In the context of the exercise to push GPT2 towards outputting reviews with more positive sentiment, we want to try and have a set of prompts that can produce varying kinds of sentiments rather than just one kind of sentiment. This set of prompts essentially forms our "observation space" and all completions are "actions", if our observation space contains primarily positive sentiment the model will not update heavily and will potentially still output negative sentiment when a prompt heavily favors it. Ideally we want our set of prompts to have a mix of sentiments.

We want to collect the first few (3-5, the choice is yours) words from each review to serve as prompts for our finetuned model. The generated text from these prompts will be later used to evaluate the performance of our finetuned model.

Emphasis - **we want to capture these prompts straight from the imdb dataset rather than write them ourselves.**

In [ ]:
def generate_prompts(dataset) -> List[str]:
    """Generate & return prompts from dataset."""
    ...


prompts = generate_prompts(imdb)
prompts[:15]

<details>
<summary>Solution</summary>


```python
def generate_prompts(dataset):
    """Generate & return prompts from dataset."""
    prompts = [" ".join(review.split()[:4]) for review in dataset["text"]]
    return prompts
```
</details>


## GPT2-IMDB

The model that we will perform RLHF on is a GPT-2 model fine-tuned on the IMDB dataset, which can be found here: https://huggingface.co/lvwerra/gpt2-imdb. Since this model is finetuned on the IMDB dataset, the distribution of sentiments of its generations will be close to the distribution of sentiments of the original dataset. This means that after fine-tuning, the responses that are categorized as "nice" tend to lean towards being positive movie reviews rather than just generically positive continuations.


### Exercise - Load the GPT-2 model and generate reviews from prompts

```c
Difficulty: 🟠🟠🟠⚪⚪
Importance: 🟠🟠🟠⚪⚪

You should spend up to 10-25 minutes on this exercise.
```

You will need to use the `AutoTokenizer` and `AutoModelForCausalLM` from the transformers package. You might want to use the `generate` method of the GPT-2 model that you load, if you do you should set the `max_new_tokens` argument to something that's large enough.

Play around with generating completions from this prompt and verify whether the completions approximately fit your initial expectations of the sentiments that the model would output.

**Note** - when you run `tokenizer(prompt)`, this will return a dictionary containing things like `token_ids` as well as a couple of other things that need to be passed into the model in a forward pass (e.g. a tensor indicating where you should mask `[PAD]` tokens). The best way to deal with this is to take `inputs = tokenizer(prompt)` and run `model.generate(**inputs)`.

In [ ]:
def generate_completion(prompt: str, model, tokenizer) -> str:
    """
    Generates completions for the given prompt (in the form of a string).

    Remember to set the `do_sample=True` flag when you call `model.generate`.
    """

    inputs = tokenizer(prompt, return_tensors="pt")
    out = model.generate(**inputs, do_sample=True)
    return tokenizer.decode(out.squeeze())


# Load the tokenizer and model.
# You can find the name of the model and tokenizer at the documentation page: https://huggingface.co/lvwerra/gpt2-imdb.
gpt2_tokenizer = ...
gpt2 = ...

generate_completion(prompts[0], gpt2, gpt2_tokenizer)

<details>
<summary>Solution</summary>

```python
def generate_completion(prompt: str, model, tokenizer) -> str:
    """
    Generates completions for the given prompt (in the form of a string).

    Remember to set the `do_sample=True` flag when you call `model.generate`.
    """

    inputs = tokenizer(prompt, return_tensors="pt")
    out_tokens = model.generate(**inputs, do_sample=True, max_new_tokens=64)
    return tokenizer.decode(out.squeeze())


# Load the tokenizer and model.
# You can find the name of the model and tokenizer at the documentation page: https://huggingface.co/lvwerra/gpt2-imdb.
gpt2_tokenizer = AutoTokenizer.from_pretrained("lvwerra/gpt2-imdb")
gpt2 = AutoModelForCausalLM.from_pretrained("lvwerra/gpt2-imdb")
```
</details>


### The reward function

Judging by the name of this chapter you might think that you would be providing the reward function yourself but sadly we will not be doing this. Instead, we will be using a language model trained to perform sentiment analysis to generate the sentiment score (higher is positive). The language model we will be using to generate sentiment scores can be found here: https://huggingface.co/lvwerra/distilbert-imdb.


#### Exercise - Get sentiment scores for a review

```c
Difficulty: 🟠🟠🟠🟠⚪
Importance: 🟠🟠🟠⚪⚪

You should spend up to 15-30 minutes on this exercise.
```

We can use the model mentioned above in eval mode to generate sentiment scores and then transform the sentiments into rewards to be fed into the RLHF training loop.

Note: Here you should use `AutoModelForSequenceClassification` instead of `AutoModelForCausalLM` since we are doing classification (what's the sentiment?) rather than generation. Do not hesitate to print the objects, shapes and types of the variable you're working with.

In [ ]:
bert_imdb = ...
bert_imdb_tokenizer = ...

@torch.inference_mode()  # Tell PyTorch to not build a computation graph and a few other things, for speed
def reward_model(samples: List[str], model=bert_imdb, tokenizer=bert_imdb_tokenizer, **kwargs) -> List[float]:
    """
    Returns the rewards for the given samples.

    kwargs are passed to your model during a forward pass.
    """

    ...

<details>
<summary>Solution</summary>


```python
bert_imdb = AutoModelForSequenceClassification.from_pretrained("lvwerra/distilbert-imdb")
bert_imdb_tokenizer = AutoTokenizer.from_pretrained("lvwerra/distilbert-imdb")

@torch.inference_mode()  # Tell PyTorch to not build a computation graph and a few other things, for speed
def reward_model(samples: List[str], model=bert_imdb, tokenizer=bert_imdb_tokenizer, **kwargs) -> List[float]:
    """
    Returns the rewards for the given samples.

    kwargs are passed to your model during a forward pass.
    """

    inputs = tokenizer(samples, padding=True, truncation=True, return_tensors="pt")

    outputs = model(**inputs, **kwargs)

    logits = outputs.logits
    probabilities = torch.softmax(logits, dim=1)

    # 1 is the index of the positive class
    return probabilities[:, 1].tolist()
```
</details>


Test your reward model on some example prompts. Do the numbers make sense?

In [ ]:
example_prompts = ["Example string", "I'm having a good day", "You are an ugly person"]
rewards = reward_model(example_prompts)

for prompt, reward in zip(example_prompts, rewards):
    print(f"{prompt}: {reward}")

# Should be around 0.54, 0.97 and 0.05

### Exercise - Output sentiment scores using Huggingface pipelines

This is an alternate way to get a reward model working directly using Huggingface pipelines. This will enable you to use a diverse range of models quite easily by changing a couple of arguments and provide you with more functionality than the vanilla PyTorch loop you implemented above. Reading the relevant documentation is the key to success here.

**Part A: Create a huggingface pipeline to output sentiment scores for a generated review**

```c
Difficulty: 🟠🟠🟠⚪⚪
Importance: 🟠🟠🟠🟠⚪

You should spend up to 10-25 minutes on this exercise.
```

Pipelines are a high-level way to use huggingface models for inference. Since the model that acts as our reward function will be used strictly for inference, it makes sense to wrap it in a pipeline.

The huggingface Pipeline documentation can be found here: [https://huggingface.co/docs/transformers/main_classes/pipelines](https://huggingface.co/docs/transformers/main_classes/pipelines).

You will need to set the `top_k` argument to the number of labels we expect the pipeline to return, in our case this would be 2 (Positive and Negative).

We would ideally also want to use the truncation flag and the batch_size argument to enable faster generation. For this exercise, these two things are not essential but could be experimented with as we will need these for later exercises.


In [ ]:
def create_pipeline(model_path):
    # Ensure we use a GPU if available
    if torch.cuda.is_available():
        device = int(os.environ.get("LOCAL_RANK", 0))
    else:
        device = -1

    return pipeline(
        "text-classification",
        model_path,
        top_k=2,
        truncation=True,
        batch_size=256,
        device=device,
    )


sentiment_fn = create_pipeline("lvwerra/distilbert-imdb")

sentiment_fn("What does the pipeline returns?")

<details>
<summary>Solution</summary>


```python
def create_pipeline(model_path):
    # Ensure we use a GPU if available
    if torch.cuda.is_available():
        device = int(os.environ.get("LOCAL_RANK", 0))
    else:
        device = -1

    return pipeline(
        "text-classification",
        model_path,
        top_k=2,
        truncation=True,
        batch_size=256,
        device=device,
    )
```
</details>


**Part B: Map the sentiment pipeline to a reward function**

```c
Difficulty: 🟠🟠🟠⚪⚪
Importance: 🟠🟠🟠🟠⚪

You should spend up to 10-20 minutes on this exercise.
```

We want the reward function to return a single number corresponding to the value of the positive label (the label we care about initially) for that generation rather than a dictionary containing the labels and their respective values (this is what the pipeline outputs, print it!).


In [ ]:
def reward_model(samples: List[str], **kwargs) -> List[float]:
    """
    Returns a list of reward values corresponding to the samples in `samples`.
    """
    ...


example_prompts = ["Example string", "I'm having a good day", "You are an ugly person"]
rewards = reward_model(example_prompts)

for prompt, reward in zip(example_prompts, rewards):
    print(f"{prompt}: {reward}")

# Should still be around 0.54, 0.97 and 0.046


<details>
<summary>Solution</summary>


```python
def reward_model(samples: List[str], **kwargs) -> List[float]:
    """
    Returns a list of reward values corresponding to the samples in `samples`.
    """

    # This is one way of doing it, but there are many others.
    return [
        result['score']
        for results in sentiment_fn(samples)
        for result in results
        if result['label'] == "POSITIVE"
    ]
```
</details>


### Exercise - Sentiment playground

```c
Difficulty: 🟠⚪⚪⚪⚪
Importance: 🟠🟠🟠🟠⚪

You should spend up to 10-15 minutes on this exercise.
```

The reward model is now ready and you should take some time to feed in sentences of varying sentiments to check whether the rewards are as you expect. Remember the reward model is also a trained model so it exhibits all the quirks of one such as weird failure modes and potential to be broken with adversarial examples.

What are the most counterintuitive results you can find? **It's vitally important for the overall experience of the exercises today that you post your findings in the Signal chat.**

We will also be using this opportunity to test whether your reward model is set up correctly.

In [ ]:
## Code below has an interesting set of examples:

prompts = [
    "I want to eat",
    "I want your puppy",
    "I want to eat your puppy",
]

for prompt in prompts:
    print(prompt, reward_model(prompt))

# 2️⃣ Using RLHF for Finetuning

> ##### Learning objectives
>
> - Learn about TRL and how it can be used
> - Use RLHF to improve sentiment of GPT2-produced movie reviews


## TRL


### What is TRL?

The trl library is a library built on top of [transformers](https://github.com/huggingface/transformers), used to fine-tune and align transformer language models using RLHF (through PPO) and [DPO](https://arxiv.org/abs/2305.18290). The trl library can also perform both supervised fine-tuning (SFT) and reward modeling.


### Using TRL

Using trl for RLHF, we need to choose:

- A model to be trained
- A RLHF trainer
- A prompt dataset.
- A reward function (which makes use of the reward model).
- Evaluation Prompts

In [ ]:
# First, we make a function for taking minibatches of strings
def make_minibatch(prompts, batch_size):
    for i in range(0, len(prompts), batch_size):
        yield prompts[i:i + batch_size]

# We use 64 prompts (generated by ChatGPT) to use as inputs in our RLHF training
# The prompts are a mix of positive, neutral, and negative sounding prompts.

prompts = ["This movie was", "I was simply", "I want to eat", "The plot was",
           "Characters felt very", "I couldn't stop", "Would definitely recommend", "The ending left",
           "Acting was just", "Cinematography seemed", "Soundtrack is a", "Scriptwriting felt",
           "Directing could have", "The pacing was", "Visual effects were", "Emotionally, it was",
           "Storyline seemed", "Loved the way", "Hated how the", "The movie overall",
           "One of the", "Could not believe", "Was pleasantly surprised", "Expected more from",
           "Definitely not worth", "Will watch again", "Could have been", "It was quite",
           "Felt like a", "Such a disappointment", "This film is", "Was really bored",
           "Truly captivating and", "Thought it would", "An absolute masterpiece", "Completely fell flat",
           "Kept me on", "The humor was", "Loved every minute", "Left me feeling",
           "The action scenes", "A total waste", "A true work", "Could watch it",
           "Not my favorite", "An enjoyable and", "Wasn't expecting it", "What a great",
           "It failed to", "The romance subplot", "Impressed by the", "Could not relate",
           "A must-see for", "Lost interest after", "The film's message", "A visual delight",
           "Unconvincing character arcs", "Absolutely thrilling", "Underwhelming performance", "Beautifully crafted",
           "I was amazed", "Really dragged on", "A solid piece", "Not engaging enough",
]


In [ ]:
# We are using again GPT-2-IMDB library, except that now wrapped in TRL

batch_size = 4
num_epochs = 100

trl_model = trl.AutoModelForCausalLMWithValueHead.from_pretrained(
    pretrained_model_name_or_path="lvwerra/gpt2-imdb",
)  # The model to be RLHFed
trl_ref_model = trl.create_reference_model(trl_model)  # The model used as a base model in KL divergence

tokenizer = AutoTokenizer.from_pretrained('lvwerra/gpt2-imdb')
tokenizer.pad_token = tokenizer.eos_token
ppo_config = trl.PPOConfig(batch_size=batch_size,
                           mini_batch_size=batch_size)
ppo_trainer = trl.PPOTrainer(ppo_config,
                             trl_model,
                             trl_ref_model,
                             tokenizer)


## Exercise: Putting it all together - Reinforcing *positive* sentiment

```c
Difficulty: 🟠🟠🟠⚪⚪
Importance: 🟠🟠🟠🟠⚪

You should spend up to 10-20 minutes on this exercise.
```

We will now be creating the training loop using TRL. The only thing you should do is to carefully understand the training loop, and what each of the moving pieces are doing. For that, we are not commenting the training loop, and you should make those comments.

In [ ]:
for epoch in range(10):
    for batch_prompts in make_minibatch(prompts, 4):
        prompts_tensor = [tokenizer.encode(prompt, return_tensors="pt").to("cuda") for prompt in batch_prompts]
        responses_tensor  = [trl.core.respond_to_batch(trl_model, prompt_tensor) for prompt_tensor in prompts_tensor]
        prompts_tensor = [p[0] for p in prompts_tensor]
        responses_tensor = [r[0] for r in responses_tensor]
        responses_strings = [tokenizer.decode(r) for r in responses_tensor]
        full_sentences = [b + r for b, r in zip(batch_prompts, responses_strings)]

        rewards = reward_model(full_sentences)
        rewards = [torch.tensor(r) for r in rewards]

        train_stats = ppo_trainer.step(prompts_tensor, responses_tensor, rewards)
    print(f'Epoch {epoch + 1} completed')

## Exercise: Sentiment playground - Post RLHF

```c
Difficulty: 🟠⚪⚪⚪⚪
Importance: 🟠🟠🟠🟠⚪

You should spend up to ~10 minutes on this exercise.
```

Try out your RLHF'd model!

In [ ]:
eval_model = ppo_trainer.model.cpu()
generate_completion("This movie", eval_model, tokenizer)

## Bonus Exercise (Hard): Now write your own PPOTrainer

```c
Difficulty: Hero Mode
Importance: 🟠🟠🟠🟠⚪

You should spend up to ~??? minutes on this exercise.
```

You've learned what RLHF returns, but now we are giving you a much harder challenge: implement the RLHF part. Namely, you are going to implement the class PPOTrainer just as in trl.PPOTrainer. Have fun!

Tips:
- Look at the [documentation for trl.PPOConfig](https://github.com/huggingface/trl/blob/main/trl/trainer/ppo_config.py), to see which configurations you can take from it (such as the KL Divergence constant value, and so on). If you find it best, you can implement your own PPOConfig class.
- Also look at the documentation for [trl.PPOTrainer](https://github.com/huggingface/trl/blob/main/trl/trainer/ppo_trainer.py), to get some inspiration. It is a really useful skill to read documentation and source code in general (and this is not a trivial task).
- Feel free to import any additional libraries you would like, look at the transformers library to understand its internals, and what not. This is an advanced coding exercise, and do not feel shy to use any tools at your hand (except for trl.PPOTrainer itself, of course).




In [ ]:
class PPOConfig(object):
    def __init__(self,
                 batch_size,
                 mini_batch_size):
        self.batch_size = batch_size
        self.mini_batch_size = mini_batch_size


class PPOTrainer(object):
    def __init__(self,
                 ppo_config,
                 model,
                 ref_model,
                 tokenizer):
        self.ppo_config = ppo_config
        self.model = model
        self.ref_model = ref_model
        self.tokenizer = tokenizer

    def step(self,
             prompts_tensor,
             responses_tensor,
             rewards):
        pass

In [ ]:
# Test your training (train it with a single epoch first):

batch_size = 4
num_epochs = 100

trl_model = trl.AutoModelForCausalLMWithValueHead.from_pretrained(
    pretrained_model_name_or_path="lvwerra/gpt2-imdb",
)  # The model to be RLHFed
trl_ref_model = trl.create_reference_model(trl_model)  # The model used as a base model in KL divergence

tokenizer = AutoTokenizer.from_pretrained('lvwerra/gpt2-imdb')
tokenizer.pad_token = tokenizer.eos_token

# If you implemented PPOConfig by yourself, use your version
ppo_config = trl.PPOConfig(batch_size=batch_size,
                           mini_batch_size=batch_size)

# Notice that we are not relying anymore on trl.PPOTrainer, but your PPOTrainer
ppo_trainer = PPOTrainer(ppo_config,
                         trl_model,
                         trl_ref_model,
                         tokenizer)


In [ ]:
for epoch in range(10):
    for batch_prompts in make_minibatch(prompts, 4):
        prompts_tensor = [tokenizer.encode(prompt, return_tensors="pt").to("cuda") for prompt in batch_prompts]
        responses_tensor  = [trl.core.respond_to_batch(trl_model, prompt_tensor) for prompt_tensor in prompts_tensor]
        prompts_tensor = [p[0] for p in prompts_tensor]
        responses_tensor = [r[0] for r in responses_tensor]
        responses_strings = [tokenizer.decode(r) for r in responses_tensor]
        full_sentences = [b + r for b, r in zip(batch_prompts, responses_strings)]

        rewards = reward_model(full_sentences)
        rewards = [torch.tensor(r) for r in rewards]

        train_stats = ppo_trainer.step(prompts_tensor, responses_tensor, rewards)
    print(f'Epoch {epoch + 1} completed')

## Exercise: Sentiment playground - Post RLHF

```c
Difficulty: 🟠⚪⚪⚪⚪
Importance: 🟠🟠🟠🟠⚪

You should spend up to ~10 minutes on this exercise.
```

Try out your RLHF'd model!


In [ ]:
eval_model = ppo_trainer.model.cpu()
generate_completion("This movie", eval_model, tokenizer)